In [ ]:
print(1+2)

3


In [9]:
ls

drive/  sample_data/


In [10]:
cd drive/

/content/drive


In [11]:
ls

MyDrive/


In [12]:
cd MyDrive/

/content/drive/MyDrive


In [13]:
ls

 1.gdoc                   Resume.docx
 balance_reviews.csv     'TurboC++ for Windows 7_v3.7.8.9major_release.exe'
'Captions Ref.gdoc'       Untitled0.ipynb
 model.h5                 Untitled1.ipynb
'project proposal.gdoc'   Untitled2.ipynb
'Resume (1).docx'        'Untitled document.gdoc'


In [14]:
import pandas as pd

df =  pd.read_csv('balance_reviews.csv',nrows=60000)



df.isnull().any(axis = 0)

#handle the missing data
df.dropna(inplace =  True)

#leaving the reviews with rating 3 and collect reviews with
#rating 1, 2, 4 and 5 onyl

df = df [df['overall'] != 3]

import numpy as np

#creating a label
#based on the values in overall column
df['Positivity'] = np.where(df['overall'] > 3 , 1 , 0)

#NLP
#reviewText - feature - df['reviewText']
#Positivity - label - df['Positivity']



#version 02
#tf-idf 
#term frequency inverse document frequency

from sklearn.model_selection import train_test_split

features_train, features_test, labels_train, labels_test = train_test_split(df['reviewText'], df['Positivity'], random_state = 42, test_size = 0.5 )



from sklearn.feature_extraction.text import TfidfVectorizer

#vect = TfidfVectorizer(min_df = 50).fit(features_train)
#vect = TfidfVectorizer(min_df=5, ngram_range=(1, 2), stop_words='english', max_features= 2000,strip_accents='unicode', norm='l2')
vect = TfidfVectorizer(min_df=5, max_features= 3000)

features_train_vectorized = vect.fit_transform(features_train)

In [15]:
len(df)

7903

In [16]:
features_train_vectorized

<3951x1766 sparse matrix of type '<class 'numpy.float64'>'
	with 86529 stored elements in Compressed Sparse Row format>

In [17]:
features_train_vectorized = features_train_vectorized.todense()

In [ ]:
#Hidden Layers : 1000, 500, 20
#IL Nodes: 3000
#OL Nodes: 1 single node with sigmoid activation

In [18]:
# Deep Learning modules

import numpy as np

from keras.models import Sequential

from keras.layers.core import Dense, Dropout, Activation

import tensorflow

from tensorflow.keras.optimizers import Adadelta, Adam, RMSprop

from keras.utils import np_utils

In [21]:
#Deep Layer Model building in Keras

#del model

model = Sequential()

model.add(Dense(1000,input_shape= (1766,)))

model.add(Activation('relu'))

model.add(Dropout(0.5))

model.add(Dense(500))

model.add(Activation('relu'))

model.add(Dropout(0.5))

model.add(Dense(50))

model.add(Activation('relu'))

model.add(Dropout(0.5))

model.add(Dense(1))

model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam')

In [22]:
model.fit(features_train_vectorized, labels_train, batch_size=100, epochs=4)

Epoch 1/4
40/40 [==============================] - 2s 30ms/step - loss: 0.5763
Epoch 2/4
40/40 [==============================] - 1s 29ms/step - loss: 0.2380
Epoch 3/4
40/40 [==============================] - 1s 30ms/step - loss: 0.1228
Epoch 4/4
40/40 [==============================] - 1s 29ms/step - loss: 0.0594


In [23]:
#Model Prediction

labels_train_predclass = model.predict(features_train_vectorized,batch_size=200)

labels_test_predclass = model.predict(vect.transform(features_test).todense(),batch_size=200)

from sklearn.metrics import accuracy_score

print(accuracy_score(labels_train, np.round(abs(labels_train_predclass)), normalize=False) )

#print(accuracy_score(labels_train, labels_test_predclass))


3931


In [24]:
model.save('model.h5')

In [25]:
#how to load the keras model
from keras.models import load_model
recreated_model = load_model('model.h5')

In [26]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 1000)              1767000   
_________________________________________________________________
activation_4 (Activation)    (None, 1000)              0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 500)               500500    
_________________________________________________________________
activation_5 (Activation)    (None, 500)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 50)               